In this file, downloded video files are cut into frames by using FFmpeg tools.

In [ ]:
#A package to use google drive
from google.colab import drive
drive.mount('/content/drive')
#'/gdrive/Shared drives/'
#https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

#Package install
!git clone https://github.com/mbuckler/youtube-bb.git /usr/local/lib/python3.7/dist-packages/youtube_bb
!pip install ffmpy
!pip install pillow
!pip install futures
!pip install youtube-dl

import imageio
import moviepy
import ffmpy
import futures
import youtube_dl
import PIL
import glob
import re
import os
import sys
import pprint
import shutil
import pandas as pd
import cv2
import numpy as np
from subprocess import call
import glob

Mounted at /content/drive
Cloning into '/usr/local/lib/python3.7/dist-packages/youtube_bb'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 67.41 KiB | 3.21 MiB/s, done.
Resolving deltas: 100% (190/190), done.
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=c4c55836dd0a2656a1bd31e0eeb66e8e1ce438977f3a83828b0e0d514059bc71
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
Successfully built ffmpy


     |████████████████████████████████| 1.9 MB 4.0 MB/s 


In [ ]:
#Class category (23 classes + none class)
class_names = ['person','bird','bicycle','boat','bus','bear','cow','cat','giraffe',
              'pottled_plant','house','motorcycle','knife','airplane','stakeboard',
              'train','truck','zebra','toilet','dog','elephant','umbrella','none','car']

#video_list container
video_list_df = pd.DataFrame(columns = ['video_file_id', 'youtube_id','class_id','class_name','object_id'])
video_list_df.video_file_id = video_list_df.video_file_id.astype(str)
video_list_df.youtube_id = video_list_df.youtube_id.astype(str)
video_list_df.class_id = video_list_df.class_id.astype(int)
video_list_df.class_name = video_list_df.class_name.astype(str)
video_list_df.object_id = video_list_df.object_id.astype(int)

k = 0 #counter
base_dir = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/' #folder where YTBB is saved

df_train = pd.read_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/youtube_boundingboxes_detection_train.csv', header=None)
df_train.columns = ['youtube_id','timestamp_ms','class_id','class_name','object_id','object_presence','xmin','xmax','ymin','ymax']

In [ ]:
#Create video list
for i in range(len(class_names)):
  print('Checking video files of ' + "class " + str(i) + " " + class_names[i] + '...')
  src_dir = base_dir + str(i) + '/'
  dir_length = len(src_dir)
  files = sorted(list(glob.glob(os.path.join(src_dir,'*.mp4')))) #create list of video files
  
  for in_path in files:
    youtube_id = in_path[(dir_length):(dir_length+11)]
    if in_path[(dir_length+12):(dir_length+14)].isnumeric(): #class_id is double digit
      class_id = int(in_path[(dir_length+12):(dir_length+14)])
      class_name = class_names[class_id] 
      if in_path[(dir_length+15):(dir_length+17)].isnumeric():
        object_id = in_path[(dir_length+15):(dir_length+17)] #object_id is double digit
        video_file_id = str(class_id) + "+" + youtube_id + "+" + str(object_id)
      else:
        object_id = int(in_path[dir_length+15]) #object_id is single digit
        video_file_id = str(class_id) + "+" + youtube_id + "+" + "0" + str(object_id)
    else: #class_id is single digit
      class_id = int(in_path[(dir_length+12):(dir_length+13)])
      class_name = class_names[class_id]
      if in_path[(dir_length+14):(dir_length+16)].isnumeric():
        object_id = int(in_path[(dir_length+14):(dir_length+16)]) #object_id is double digit
        video_file_id = str(class_id) + "+" + youtube_id + "+" + "0" + str(object_id)
      else:
        object_id = int(in_path[dir_length+14]) #object_id is single digit
        video_file_id = "0" + str(class_id) + "+" + youtube_id + "+" + "0" + str(object_id)
    
    video_list_df.loc[k] = [video_file_id, youtube_id, class_id, class_name, object_id]
    k += 1

Checking video files of class 0 person...
Checking video files of class 1 bird...
Checking video files of class 2 bicycle...
Checking video files of class 3 boat...
Checking video files of class 4 bus...
Checking video files of class 5 bear...
Checking video files of class 6 cow...
Checking video files of class 7 cat...
Checking video files of class 8 giraffe...
Checking video files of class 9 pottled_plant...
Checking video files of class 10 house...
Checking video files of class 11 motorcycle...
Checking video files of class 12 knife...
Checking video files of class 13 airplane...
Checking video files of class 14 stakeboard...
Checking video files of class 15 train...
Checking video files of class 16 truck...
Checking video files of class 17 zebra...
Checking video files of class 18 toilet...
Checking video files of class 19 dog...
Checking video files of class 20 elephant...
Checking video files of class 21 umbrella...
Checking video files of class 22 none...
Checking video files of

In [ ]:
video_list_df.to_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/video_list.csv')
print(video_list_df)

           video_file_id   youtube_id  class_id class_name  object_id
0      00+--0bLFuriZ4+00  --0bLFuriZ4         0     person          0
1      00+--4VWx_0Sc4+00  --4VWx_0Sc4         0     person          0
2      00+--7KXqgihyw+00  --7KXqgihyw         0     person          0
3      00+--7L39R9NG0+00  --7L39R9NG0         0     person          0
4      00+--BYHGryxto+00  --BYHGryxto         0     person          0
...                  ...          ...       ...        ...        ...
18554  23+3qDnJUD7wLs+00  3qDnJUD7wLs        23        car          0
18555  23+3qa2D_Q7CJI+00  3qa2D_Q7CJI        23        car          0
18556  23+3rLi9Qu_iqc+00  3rLi9Qu_iqc        23        car          0
18557  23+3sbpTPaordY+00  3sbpTPaordY        23        car          0
18558  23+3smGU4e12QA+00  3smGU4e12QA        23        car          0

[18559 rows x 5 columns]


In [ ]:
#Slice videos into images
output_base_dir = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/' #folder where sliced images will be saved

for i in range(len(class_names)):
  os.mkdir(output_base_dir + str(i)) #folder where sliced images of a class will be saved
  output_dir = output_base_dir + str(i) #folder where sliced images of a class will be saved
  video_dir = base_dir + str(i) #foler where videos are saved
  video_files = sorted(list(glob.glob(os.path.join(video_dir,'*.mp4'))))

  video_list_df_sub = video_list_df[video_list_df.class_id == i] #extract video list of class i
  video_list_df_sub = video_list_df_sub.reset_index(drop=True) #reset the index in the video list
  k = 0 #counter in each video list

  for in_path in video_files:
    video_file_id = video_list_df_sub.iloc[k]['video_file_id'] #extract video file id from the video list
    out_path = os.path.join(output_dir,'{}+%06d.jpg'.format(video_file_id)) #name slice images
    call(['ffmpeg','-i', in_path, '-vf', 'fps=1', out_path]) #slicing a video by using ffmpeg
    print('Completed slicing '+ str(k+1) + ' / ' + str(len(video_files)) + ' of class ' + str(i) + ' ' + class_names[i] + ' images')
    k += 1

Streaming output truncated to the last 5000 lines.
Completed slicing 352 / 535 of class 14 stakeboard images
Completed slicing 353 / 535 of class 14 stakeboard images
Completed slicing 354 / 535 of class 14 stakeboard images
Completed slicing 355 / 535 of class 14 stakeboard images
Completed slicing 356 / 535 of class 14 stakeboard images
Completed slicing 357 / 535 of class 14 stakeboard images
Completed slicing 358 / 535 of class 14 stakeboard images
Completed slicing 359 / 535 of class 14 stakeboard images
Completed slicing 360 / 535 of class 14 stakeboard images
Completed slicing 361 / 535 of class 14 stakeboard images
Completed slicing 362 / 535 of class 14 stakeboard images
Completed slicing 363 / 535 of class 14 stakeboard images
Completed slicing 364 / 535 of class 14 stakeboard images
Completed slicing 365 / 535 of class 14 stakeboard images
Completed slicing 366 / 535 of class 14 stakeboard images
Completed slicing 367 / 535 of class 14 stakeboard images
Completed slicing 368

In [ ]:
#Edit image file names and create metadata for the later process
class_names = ['person','bird','bicycle','boat','bus','bear','cow','cat','giraffe',
              'pottled_plant','house','motorcycle','knife','airplane','stakeboard',
              'train','truck','zebra','toilet','dog','elephant','umbrella','none','car']
base_dir = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/'
input_base_dir = base_dir + 'sliced_images' #folder name where sliced images are saved
output_base_dir = base_dir + 'sliced_images_revised' #folder name where sliced images with revised names are saved
metadata_df = pd.DataFrame(columns = ['ImageID','Xmin','Ymin','Xmax','Ymax','Width','Height','ClassName','ObjectPresent'])
k = 0 #counter in all image files

#target_class = list(range(0,24))
#Due to the process workload across 24 classes, 7 notebook files are run parallely. 
#To reduce the redundancy in the submission, only this file is submitted. 
#This file includes output of class 12 knife and class 13 airplane

#target_class = list(range(0,24))
target_class = list(range(12,24))

for l in target_class:
  #create output folder
  if len(str(l)) == 1:
    class_num = '0' + str(l)
  else:
    class_num = str(l)
  os.mkdir(output_base_dir + '/' + class_num + '_' + class_names[l])

  sub_video_list = sorted(video_list_df[video_list_df.class_id == l]['video_file_id'])
  image_files = sorted(list(glob.glob(input_base_dir + '/' + str(l) + '/' + '*.jpg'))) #This process take time
  d = 1 #counter for progress check (count of image files in a class)
  c = 1 #counter by number of images by video. Start utilisation of original meta data from the second row because ffmpeg cannot capture timestamp = 0
  previous_video_file_id = '' #blank for the initial value

  for j in image_files:
    #Extract necessary data from sliced image file names to create to update the annotation data
    if len(str(l)) == 1:
      youtube_id = j[(len(input_base_dir)+6):(len(input_base_dir)+17)] 
      object_id = int(j[(len(input_base_dir)+18):(len(input_base_dir)+20)])
      video_file_id = j[(len(input_base_dir)+3):(len(input_base_dir)+20)]
    else:
      youtube_id = j[(len(input_base_dir)+7):(len(input_base_dir)+18)]
      object_id = int(j[(len(input_base_dir)+19):(len(input_base_dir)+21)])
      video_file_id = j[(len(input_base_dir)+4):(len(input_base_dir)+21)]

    #df_train is the original annotation file provided by YTBB
    #df_train_sub is the specific class annotation data extracted from df_train
    df_train_sub = df_train[(df_train.youtube_id == youtube_id) & (df_train.object_id == object_id) & (df_train.class_id == l)]
    df_train_sub = df_train_sub.reset_index(drop=True)

    if video_file_id != previous_video_file_id:
      c = 1 #recent counter by number of images by video

    if c >= df_train_sub.shape[0]: #if metadata does not exist, skip the image file
      print('No metadata for the image')
      c += 1 #counter by number of images by video
      continue

    if df_train_sub.object_presence[c] == 'present':
      ObjectPresent = 1 #present 
    elif df_train_sub.object_presence[c] == 'absent':
      ObjectPresent = 0 #absent
    else:
      ObjectPresent = 2 #uncertain
      
    if len(str(df_train_sub.timestamp_ms[c])) == 7:
      timestamp = str(df_train_sub.timestamp_ms[c])
    else:
      timestamp = '0'*(7-len(str(df_train_sub.timestamp_ms[c]))) + str(df_train_sub.timestamp_ms[c])
    
    #Create new annotation file appropriate for the model training
    ImageID = video_file_id + '+' + timestamp + '+' + str(ObjectPresent) + '.jpg' #change
    Xmin = df_train_sub.xmin[c]
    Ymin = df_train_sub.ymin[c]
    Xmax = df_train_sub.xmax[c]
    Ymax = df_train_sub.ymax[c]
    Width = df_train_sub.xmax[c] - df_train_sub.xmin[c]
    Height = df_train_sub.ymax[c] - df_train_sub.ymin[c]
    ClassName = class_names[l] #df_train_sub.class_name[c] is error
      
    metadata_df.loc[k] = [ImageID, Xmin, Ymin, Xmax, Ymax, Width, Height, ClassName, ObjectPresent]
    shutil.copy(j, output_base_dir + '/' + class_num + '_' + class_names[l] + '/' + ImageID)
    k += 1 #counter in all image files
    c += 1 #counter by number of images by video

    previous_video_file_id = video_file_id #use this to reset counter c
    print('Changing file name of ' + str(d) + ' / ' + str(len(image_files)) + ' of class ' + str(l) + ' ' + class_names[l]) 
    d += 1 #counter by number of images in a class

Streaming output truncated to the last 5000 lines.
Changing file name of 7320 / 12387 of class 13 airplane
Changing file name of 7321 / 12387 of class 13 airplane
Changing file name of 7322 / 12387 of class 13 airplane
Changing file name of 7323 / 12387 of class 13 airplane
Changing file name of 7324 / 12387 of class 13 airplane
Changing file name of 7325 / 12387 of class 13 airplane
Changing file name of 7326 / 12387 of class 13 airplane
Changing file name of 7327 / 12387 of class 13 airplane
Changing file name of 7328 / 12387 of class 13 airplane
Changing file name of 7329 / 12387 of class 13 airplane
Changing file name of 7330 / 12387 of class 13 airplane
Changing file name of 7331 / 12387 of class 13 airplane
Changing file name of 7332 / 12387 of class 13 airplane
Changing file name of 7333 / 12387 of class 13 airplane
Changing file name of 7334 / 12387 of class 13 airplane
Changing file name of 7335 / 12387 of class 13 airplane
Changing file name of 7336 / 12387 of class 13 airpla

FileExistsError: ignored